# Введение в перцептрон и нейронные сети

В этой тетрадке мы познакомимся с основами перцептрона и нейронных сетей, реализуем простой перцептрон и обучим его на классических задачах.

## 1. Что такое перцептрон?

**Перцептрон** — это простейшая модель искусственного нейрона, предложенная Фрэнком Розенблаттом в 1957 году.

### Математическая модель:

Перцептрон вычисляет взвешенную сумму входов и применяет к ней функцию активации:

$$y = f\left(\sum_{i=1}^{n} w_i x_i + b\right)$$

где:
- $x_i$ — входные признаки
- $w_i$ — веса
- $b$ — смещение (bias)
- $f$ — функция активации (обычно ступенчатая)

### Функция активации:

Классическая ступенчатая функция:
$$f(z) = \begin{cases} 1, & \text{если } z \geq 0 \\ 0, & \text{если } z < 0 \end{cases}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, make_circles
from sklearn.model_selection import train_test_split

# Устанавливаем seed для воспроизводимости
np.random.seed(42)

## 2. Реализация перцептрона

In [ ]:
class Perceptron:
    """Простая реализация перцептрона"""
    
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None
        self.errors_ = []
    
    def activation(self, z):
        """Ступенчатая функция активации"""
        return np.where(z >= 0, 1, 0)
    
    def fit(self, X, y):
        """Обучение перцептрона"""
        n_samples, n_features = X.shape
        
        # Инициализация весов и смещения
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        # Обучение
        for _ in range(self.n_iterations):
            errors = 0
            for xi, target in zip(X, y):
                # Предсказание
                linear_output = np.dot(xi, self.weights) + self.bias
                y_pred = self.activation(linear_output)
                
                # Обновление весов
                update = self.learning_rate * (target - y_pred)
                self.weights += update * xi
                self.bias += update
                
                # Подсчет ошибок
                errors += int(update != 0.0)
            
            self.errors_.append(errors)
            
            # Остановка, если нет ошибок
            if errors == 0:
                break
        
        return self
    
    def predict(self, X):
        """Предсказание классов"""
        linear_output = np.dot(X, self.weights) + self.bias
        return self.activation(linear_output)

## 3. Обучение на логических функциях

Проверим работу перцептрона на простых логических функциях.

### 3.1. Функция AND

In [ ]:
# Данные для AND
X_and = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_and = np.array([0, 0, 0, 1])

# Обучение
perceptron_and = Perceptron(learning_rate=0.1, n_iterations=10)
perceptron_and.fit(X_and, y_and)

# Предсказания
predictions = perceptron_and.predict(X_and)

print("Функция AND:")
print(f"Входы: {X_and}")
print(f"Ожидаемые выходы: {y_and}")
print(f"Предсказания: {predictions}")
print(f"Точность: {np.mean(predictions == y_and) * 100:.2f}%")

### 3.2. Функция OR

In [ ]:
# Данные для OR
X_or = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_or = np.array([0, 1, 1, 1])

# Обучение
perceptron_or = Perceptron(learning_rate=0.1, n_iterations=10)
perceptron_or.fit(X_or, y_or)

# Предсказания
predictions = perceptron_or.predict(X_or)

print("Функция OR:")
print(f"Входы: {X_or}")
print(f"Ожидаемые выходы: {y_or}")
print(f"Предсказания: {predictions}")
print(f"Точность: {np.mean(predictions == y_or) * 100:.2f}%")

### 3.3. Функция XOR (проблема линейной неразделимости)

Классическая проблема перцептрона — невозможность решить задачу XOR, так как данные линейно неразделимы.

In [ ]:
# Данные для XOR
X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_xor = np.array([0, 1, 1, 0])

# Обучение
perceptron_xor = Perceptron(learning_rate=0.1, n_iterations=100)
perceptron_xor.fit(X_xor, y_xor)

# Предсказания
predictions = perceptron_xor.predict(X_xor)

print("Функция XOR:")
print(f"Входы: {X_xor}")
print(f"Ожидаемые выходы: {y_xor}")
print(f"Предсказания: {predictions}")
print(f"Точность: {np.mean(predictions == y_xor) * 100:.2f}%")
print("\n⚠️ Перцептрон не может решить XOR! Нужна многослойная сеть.")

## 4. Визуализация разделяющей прямой

In [ ]:
def plot_decision_boundary(perceptron, X, y, title):
    """Визуализация разделяющей прямой"""
    plt.figure(figsize=(8, 6))
    
    # Точки данных
    plt.scatter(X[y == 0, 0], X[y == 0, 1], c='red', marker='o', s=100, label='Класс 0')
    plt.scatter(X[y == 1, 0], X[y == 1, 1], c='blue', marker='s', s=100, label='Класс 1')
    
    # Разделяющая прямая
    x_min, x_max = -0.5, 1.5
    if perceptron.weights[1] != 0:
        x_line = np.array([x_min, x_max])
        y_line = -(perceptron.weights[0] * x_line + perceptron.bias) / perceptron.weights[1]
        plt.plot(x_line, y_line, 'g--', linewidth=2, label='Разделяющая прямая')
    
    plt.xlim(x_min, x_max)
    plt.ylim(-0.5, 1.5)
    plt.xlabel('x₁')
    plt.ylabel('x₂')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# Визуализация для AND, OR, XOR
plot_decision_boundary(perceptron_and, X_and, y_and, 'Перцептрон: AND')
plot_decision_boundary(perceptron_or, X_or, y_or, 'Перцептрон: OR')
plot_decision_boundary(perceptron_xor, X_xor, y_xor, 'Перцептрон: XOR (неудача)')

## 5. Многослойный перцептрон (MLP)

Для решения линейно неразделимых задач (как XOR) нужна многослойная сеть с нелинейными функциями активации.

In [ ]:
class MLP:
    """Простой многослойный перцептрон с одним скрытым слоем"""
    
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.1):
        # Инициализация весов
        self.W1 = np.random.randn(input_size, hidden_size) * 0.5
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.5
        self.b2 = np.zeros((1, output_size))
        self.learning_rate = learning_rate
    
    def sigmoid(self, z):
        """Сигмоидальная функция активации"""
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def sigmoid_derivative(self, z):
        """Производная сигмоиды"""
        s = self.sigmoid(z)
        return s * (1 - s)
    
    def forward(self, X):
        """Прямое распространение"""
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)
        return self.a2
    
    def backward(self, X, y, output):
        """Обратное распространение ошибки"""
        m = X.shape[0]
        
        # Ошибка выходного слоя
        dz2 = output - y.reshape(-1, 1)
        dW2 = np.dot(self.a1.T, dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
        
        # Ошибка скрытого слоя
        dz1 = np.dot(dz2, self.W2.T) * self.sigmoid_derivative(self.z1)
        dW1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m
        
        # Обновление весов
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2
        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1
    
    def train(self, X, y, epochs=10000):
        """Обучение сети"""
        losses = []
        for epoch in range(epochs):
            # Прямое распространение
            output = self.forward(X)
            
            # Вычисление ошибки
            loss = np.mean((output - y.reshape(-1, 1)) ** 2)
            losses.append(loss)
            
            # Обратное распространение
            self.backward(X, y, output)
            
            if epoch % 1000 == 0:
                print(f'Epoch {epoch}, Loss: {loss:.6f}')
        
        return losses
    
    def predict(self, X):
        """Предсказание классов"""
        output = self.forward(X)
        return (output > 0.5).astype(int).flatten()

## 6. Решение XOR с помощью MLP

In [ ]:
# Создание и обучение MLP для XOR
mlp = MLP(input_size=2, hidden_size=4, output_size=1, learning_rate=0.5)
losses = mlp.train(X_xor, y_xor, epochs=5000)

# Предсказания
predictions = mlp.predict(X_xor)

print("\nРезультаты MLP на XOR:")
print(f"Входы: {X_xor}")
print(f"Ожидаемые выходы: {y_xor}")
print(f"Предсказания: {predictions}")
print(f"Точность: {np.mean(predictions == y_xor) * 100:.2f}%")

## 7. Визуализация обучения

In [ ]:
# График ошибки обучения
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.xlabel('Эпоха')
plt.ylabel('MSE Loss')
plt.title('Кривая обучения MLP на задаче XOR')
plt.grid(True, alpha=0.3)
plt.show()

## 8. Применение на реальных данных

Протестируем MLP на более сложных данных.

In [ ]:
# Генерация линейно разделимых данных
X_linear, y_linear = make_classification(
    n_samples=200,
    n_features=2,
    n_redundant=0,
    n_informative=2,
    n_clusters_per_class=1,
    flip_y=0.1,
    random_state=42
)

# Визуализация
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_linear[y_linear == 0, 0], X_linear[y_linear == 0, 1], 
            c='red', marker='o', alpha=0.6, label='Класс 0')
plt.scatter(X_linear[y_linear == 1, 0], X_linear[y_linear == 1, 1], 
            c='blue', marker='s', alpha=0.6, label='Класс 1')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.title('Линейно разделимые данные')
plt.legend()
plt.grid(True, alpha=0.3)

# Генерация нелинейно разделимых данных (окружности)
X_circles, y_circles = make_circles(n_samples=200, noise=0.1, factor=0.5, random_state=42)

plt.subplot(1, 2, 2)
plt.scatter(X_circles[y_circles == 0, 0], X_circles[y_circles == 0, 1], 
            c='red', marker='o', alpha=0.6, label='Класс 0')
plt.scatter(X_circles[y_circles == 1, 0], X_circles[y_circles == 1, 1], 
            c='blue', marker='s', alpha=0.6, label='Класс 1')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.title('Нелинейно разделимые данные (окружности)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Обучение на линейно разделимых данных
X_train, X_test, y_train, y_test = train_test_split(
    X_linear, y_linear, test_size=0.3, random_state=42
)

mlp_linear = MLP(input_size=2, hidden_size=5, output_size=1, learning_rate=0.1)
losses_linear = mlp_linear.train(X_train, y_train, epochs=1000)

# Оценка
train_pred = mlp_linear.predict(X_train)
test_pred = mlp_linear.predict(X_test)

print(f"\nЛинейно разделимые данные:")
print(f"Точность на обучающей выборке: {np.mean(train_pred == y_train) * 100:.2f}%")
print(f"Точность на тестовой выборке: {np.mean(test_pred == y_test) * 100:.2f}%")

In [ ]:
# Обучение на нелинейно разделимых данных
X_train, X_test, y_train, y_test = train_test_split(
    X_circles, y_circles, test_size=0.3, random_state=42
)

mlp_circles = MLP(input_size=2, hidden_size=10, output_size=1, learning_rate=0.5)
losses_circles = mlp_circles.train(X_train, y_train, epochs=5000)

# Оценка
train_pred = mlp_circles.predict(X_train)
test_pred = mlp_circles.predict(X_test)

print(f"\nНелинейно разделимые данные (окружности):")
print(f"Точность на обучающей выборке: {np.mean(train_pred == y_train) * 100:.2f}%")
print(f"Точность на тестовой выборке: {np.mean(test_pred == y_test) * 100:.2f}%")

## 9. Ключевые выводы

### Перцептрон:
- ✅ Прост в реализации и обучении
- ✅ Гарантированно сходится для линейно разделимых данных
- ❌ Не может решать линейно неразделимые задачи (XOR)
- ❌ Ограничен бинарной классификацией

### Многослойный перцептрон (MLP):
- ✅ Может решать нелинейные задачи
- ✅ Универсальный аппроксиматор (может приблизить любую функцию)
- ✅ Подходит для многоклассовой классификации
- ⚠️ Требует больше вычислительных ресурсов
- ⚠️ Может переобучаться
- ⚠️ Требует настройки гиперпараметров

### Основные концепции:
1. **Веса и смещения** — обучаемые параметры модели
2. **Функция активации** — добавляет нелинейность
3. **Прямое распространение** — вычисление выхода сети
4. **Обратное распространение** — обновление весов на основе ошибки
5. **Градиентный спуск** — алгоритм оптимизации

### Рекомендации:
- Начинайте с простых моделей (перцептрон)
- Переходите к MLP при линейной неразделимости
- Нормализуйте входные данные
- Используйте валидацию для предотвращения переобучения
- Экспериментируйте с архитектурой и гиперпараметрами

## 10. Задания для самостоятельной работы

1. Реализуйте функцию активации ReLU и сравните результаты с сигмоидой
2. Добавьте визуализацию границы решений для MLP
3. Реализуйте регуляризацию (L2) для предотвращения переобучения
4. Создайте MLP с несколькими скрытыми слоями
5. Реализуйте метод Adam для оптимизации вместо простого градиентного спуска
6. Протестируйте на датасете Iris или Wine
7. Сравните свою реализацию с MLPClassifier из sklearn